# Generate automated session reports for all participants

In [1]:
# import packages
import pandas as pd
import numpy as np
import mne
import mne_bids
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"
import q1k_init_tools as qit
import glob
import warnings
from nbconvert import HTMLExporter
from nbconvert.preprocessors import ExecutePreprocessor
from nbformat import read
import os
import re
import papermill as pm
warnings.filterwarnings('ignore')

### Select task parameters and file paths

In [2]:
# Select kind of data experimental or control group

dataset_group = "experimental"

#if dataset_group == "control":
#    # Control group data
#    project_path = "/home/james/q1k/pilot/q1k-external-pilot/"
#    task_id_in = "ap"
#    task_id_in_et = "ap"
#    task_id_out = "ap"
#    #subject_id = '002'
#    session_id = '01'
#    run_id = '1'

#elif dataset_group == "experimental":
# Experimental group data

project_path = "/project/def-emayada/q1k/experimental/"
code_path = "code/q1k_eeget_init/"
task_id_in = "TO"
task_id_in_et = "TO"
task_id_out = "TO"
#subject_id = 'Q1K_HSJ_100123_F1'
run_id = '1'
session_id = '01'
site_code = 'HSJ' #'MHC' or 'HSJ'
sourcedata_path = "sourcedata/" 
html_reports_path = "session_reports/"
#et_sync = False


In [3]:
# Sanity check to see which task you would like to make reports for 
print(task_id_in)
#et_sync

TO


In [4]:
glob.glob(project_path + code_path + html_reports_path + task_id_in + "/*.html")

['/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/1112M1_TO.html',
 '/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/0200M1_TO.html',
 '/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/0200S1_TO.html',
 '/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/1073M1_TO.html',
 '/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/1106F1_TO.html',
 '/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/0171F1_TO.html',
 '/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/0154P_TO.html',
 '/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/1118P_TO.html',
 '/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/1073P_TO.html',
 '/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/1140P_TO.html',
 '/project/def-emayada/q1k/experimental/code

In [5]:
# Generate list of session reports already processed so they are not processed again in next step
processed_sessions = []

for subject in glob.glob(project_path + code_path + html_reports_path + task_id_in + "/*.html"): 
    html_file = subject.split('/')[-1]
    processed_sessions.append(html_file.split('_')[:1])
    print(subject)

print('Existing session reports:')
processed_sessions

/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/1112M1_TO.html
/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/0200M1_TO.html
/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/0200S1_TO.html
/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/1073M1_TO.html
/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/1106F1_TO.html
/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/0171F1_TO.html
/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/0154P_TO.html
/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/1118P_TO.html
/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/1073P_TO.html
/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/1140P_TO.html
/project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/1028S1_

[['1112M1'],
 ['0200M1'],
 ['0200S1'],
 ['1073M1'],
 ['1106F1'],
 ['0171F1'],
 ['0154P'],
 ['1118P'],
 ['1073P'],
 ['1140P'],
 ['1028S1'],
 ['0050P'],
 ['1001M1'],
 ['1061F1'],
 ['1001S1'],
 ['1162M1'],
 ['0216P'],
 ['1147P'],
 ['1114M1'],
 ['0104P'],
 ['1033P'],
 ['0162M1'],
 ['1061S3'],
 ['0162S1'],
 ['0196S2'],
 ['1102S2'],
 ['0181M1'],
 ['1109P'],
 ['0200P'],
 ['1106P'],
 ['1080F1'],
 ['1045F1'],
 ['1052S1'],
 ['1052M1'],
 ['0248P'],
 ['1024M1'],
 ['0183S1'],
 ['0183M1'],
 ['0223M1'],
 ['1042P'],
 ['0050M1'],
 ['1147S1'],
 ['0119M1'],
 ['0281P'],
 ['1118M1'],
 ['1052P'],
 ['0183P'],
 ['1083F1'],
 ['1026M1'],
 ['1121F1'],
 ['0068F1'],
 ['0218P'],
 ['1009M1'],
 ['1109F1'],
 ['1093P'],
 ['1057M1'],
 ['1037F1'],
 ['1009S1'],
 ['1101P'],
 ['1140S1'],
 ['1083P'],
 ['1093S1'],
 ['1143S1'],
 ['0162P'],
 ['1121P'],
 ['1042F1'],
 ['1045P'],
 ['0186M1'],
 ['1134M1'],
 ['0043F1'],
 ['1134S1'],
 ['1083S1'],
 ['1118F1'],
 ['1078F1'],
 ['0104C1'],
 ['1001P'],
 ['0179M1'],
 ['1147F1'],
 ['1028S2']

## Generate html session reports for all participants

In [6]:
glob.glob(project_path + sourcedata_path + site_code + "/eeg/Q1K*/Q1K*" + task_id_in + '_*.mff')

['/project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1147_F1/Q1K_HSJ_1525-1147_F1_TO_20250122_122346.mff',
 '/project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1118_F1/Q1K_HSJ_1525-1118_F1_TO_20241129_101900.mff',
 '/project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1093_S2/Q1K_HSJ_1525-1093_S1_TO_20241213_114102.mff',
 '/project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_100128_P/Q1K_HSJ_100128_P_TO_20240705_113842.mff',
 '/project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1042_F1/Q1K_HSJ_1025-1042_F1_TO_20240715_092725.mff',
 '/project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1121_F1/Q1K_HSJ_1525-1121_F1_TO_20241121_022113.mff',
 '/project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1083_S1/Q1K_HSJ_1525-1083_S1_TO_20241004_105530.mff',
 '/project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_10043_F1/Q1K_HSJ_10043_F1_TO_20240531_013609.mff',
 '/proje

In [7]:
# Make sure the output directory exists
if not os.path.exists(html_reports_path):
    os.makedirs(html_reports_path)

# Create a list of sessions with errors
error_subjects = []

#Loop through existing sessions and execute the q1k_generate_individual_reports.ipynb if a session report does not already exist
#for file in glob.glob(project_path + sourcedata_path + "eeg/Q1K*/Q1K*" + task_id_in + '_*.mff'):
#    print('Current data file: ' + file)
#    # Select anything after the Q1K and before the AEP
#    #subject_id = file.split('_')[2]
#    site_part = file.split(site_code + "_")
#    subject_number = site_part[1].split("_")[0]
#    print('Participant number: ' + subject_number)
#   subject_relation = site_part[1].split("_")[1].split("/")[0]
#    print('Participant relation: ' + subject_relation)

#    subject_id_in = subject_number + "_" + subject_relation
#    print('Participant ID input: ' + subject_id_in)
#    subject_id_out = subject_number.replace('_','').replace('-','') + subject_relation
#    print('Participant ID output: ' + subject_id_out)
    
for file in glob.glob(project_path + sourcedata_path + site_code + "/eeg/Q1K*/Q1K*" + task_id_in + '_*.mff'):
    print('Current data file: ' + file)
    # Select anything after the Q1K and before the AEP
    #subject_id = file.split('_')[2]
    site_part = file.split(site_code + "_")
    subject_number = site_part[1].split("_")[0]
    print('Participant number: ' + subject_number)
    #subject_relation = site_part[1].split("_")[1].split("/")[0]
    parts = re.split(r'_{1,2}', site_part[1])  # Splits on either _ or __
    if len(parts) > 1:
        subject_relation = parts[1].split("/")[0]  # Extract the second part and split on '/'
    else:
        subject_relation = None  # Handle the case where there's no second part
    print('Participant relation: ' + subject_relation)
    
    family_code_out=qit.set_family_code(subject_number)
    print('family code out: ' + family_code_out)

    subject_id_in = subject_number + "_" + subject_relation
    print('Participant ID input: ' + subject_id_in)
    #subject_id_out = subject_number.replace('_','').replace('-','') + subject_relation
    subject_id_out = family_code_out + subject_relation
    print('Participant ID output: ' + subject_id_out)

    # Skip sessions that have a session report in the output directory
    print(subject_id_out)
    #if subject_id_out in processed_sessions:
    if any(subject_id_out in session for session in processed_sessions):
        print(subject_id_out + ' has already been processed')
        continue    

    # Define paths
    input_notebook = project_path + code_path +'q1k_generate_individual_reports.ipynb'
    print('Input notebook: ' + input_notebook)

    # Make sure the directory exists
    if not os.path.exists(f'{project_path}{code_path}session_reports/{task_id_in}/executed_notebooks/'):
        os.makedirs(f'{project_path}{code_path}session_reports/{task_id_in}/executed_notebooks/')
    
    output_notebook = f'{project_path}{code_path}{html_reports_path}{task_id_out}/executed_notebooks/{subject_id_out}_{task_id_out}_executed.ipynb'
    print('Output notebook file: ' + output_notebook)
    output_html = f'{project_path}{code_path}{html_reports_path}{task_id_out}/{subject_id_out}_{task_id_out}.html'
    print('Output HTML file: ' + output_html)

    try:
        # Execute the notebook
        pm.execute_notebook(input_notebook, output_notebook, kernel_name = 'q1k_env', parameters=dict(subject_id_in=subject_id_in, subject_id_out=subject_id_out,task_id_in=task_id_in, task_id_in_et=task_id_in_et,
         task_id_out=task_id_out, run_id=run_id, session_id=session_id, project_path=project_path, dataset_group=dataset_group, site_code=site_code))

        # Convert executed notebook to HTML
        html_exporter = HTMLExporter()
        html_exporter.exclude_input = True

        (body, resources) = html_exporter.from_filename(output_notebook)

        # Save HTML output
        with open(output_html, 'w', encoding='utf-8') as f:
            f.write(body)

        print(f"HTML report saved for {subject_id_in}.")
    

    except Exception as e:
        # Handle the error 
        error_subjects.append(subject_id_in)
        print(f"Error while processing {subject_id_in}: {e}")

# Print out the list of subjects with errors
print( "These subjects have errors: " + str(error_subjects) + " and need to be reprocessed")

Current data file: /project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1147_F1/Q1K_HSJ_1525-1147_F1_TO_20250122_122346.mff
Participant number: 1525-1147
Participant relation: F1
family code out: 1147
Participant ID input: 1525-1147_F1
Participant ID output: 1147F1
1147F1
1147F1 has already been processed
Current data file: /project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1118_F1/Q1K_HSJ_1525-1118_F1_TO_20241129_101900.mff
Participant number: 1525-1118
Participant relation: F1
family code out: 1118
Participant ID input: 1525-1118_F1
Participant ID output: 1118F1
1118F1
1118F1 has already been processed
Current data file: /project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1093_S2/Q1K_HSJ_1525-1093_S1_TO_20241213_114102.mff
Participant number: 1525-1093
Participant relation: S2
family code out: 1093
Participant ID input: 1525-1093_S2
Participant ID output: 1093S2
1093S2
1093S2 has already been processed
Current data file: /project/

Executing:   0%|          | 0/56 [00:00<?, ?cell/s]

HTML report saved for 1525-1006_M1.
Current data file: /project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1102_S2/Q1K_HSJ_1525-1102_S2_TO_20241008_015508.mff
Participant number: 1525-1102
Participant relation: S2
family code out: 1102
Participant ID input: 1525-1102_S2
Participant ID output: 1102S2
1102S2
1102S2 has already been processed
Current data file: /project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1001_S1/Q1K_HSJ_1525-1001_S1_TO_20240524_094020.mff
Participant number: 1525-1001
Participant relation: S1
family code out: 1001
Participant ID input: 1525-1001_S1
Participant ID output: 1001S1
1001S1
1001S1 has already been processed
Current data file: /project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1080_F1/Q1K_HSJ_1525-1080_F1_TO_20240823_125707.mff
Participant number: 1525-1080
Participant relation: F1
family code out: 1080
Participant ID input: 1525-1080_F1
Participant ID output: 1080F1
1080F1
1080F1 has already been pr

Executing:   0%|          | 0/56 [00:00<?, ?cell/s]

HTML report saved for 1025-1042_M1.
Current data file: /project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1037_P/Q1K_HSJ_1525-1037_P_TO_2_20240619_113129.mff
Participant number: 1525-1037
Participant relation: P
family code out: 1037
Participant ID input: 1525-1037_P
Participant ID output: 1037P
1037P
1037P has already been processed
Current data file: /project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1037_P/Q1K_HSJ_1525-1037_P_TO_20240619_112523.mff
Participant number: 1525-1037
Participant relation: P
family code out: 1037
Participant ID input: 1525-1037_P
Participant ID output: 1037P
1037P
1037P has already been processed
Current data file: /project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1057_M1/Q1K_HSJ_1525-1057_M1_TO_20240722_021517.mff
Participant number: 1525-1057
Participant relation: M1
family code out: 1057
Participant ID input: 1525-1057_M1
Participant ID output: 1057M1
1057M1
1057M1 has already been processed
Curr

Executing:   0%|          | 0/56 [00:00<?, ?cell/s]

HTML report saved for 1525-1147_S2.
Current data file: /project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1025-1061_M/Q1K_HSJ_1025-1061_M_TO_20240718_023831.mff
Participant number: 1025-1061
Participant relation: M
family code out: 1061
Participant ID input: 1025-1061_M
Participant ID output: 1061M
1061M
1061M has already been processed
Current data file: /project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1114_P/Q1K_HSJ_1525-1114_P_TO_20241120_094506.mff
Participant number: 1525-1114
Participant relation: P
family code out: 1114
Participant ID input: 1525-1114_P
Participant ID output: 1114P
1114P
Input notebook: /project/def-emayada/q1k/experimental/code/q1k_eeget_init/q1k_generate_individual_reports.ipynb
Output notebook file: /project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/executed_notebooks/1114P_TO_executed.ipynb
Output HTML file: /project/def-emayada/q1k/experimental/code/q1k_eeget_init/session_reports/TO/1114P_TO.html


Executing:   0%|          | 0/56 [00:00<?, ?cell/s]

HTML report saved for 1525-1114_P.
Current data file: /project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1024_M1/Q1K_HSJ_1525-1024_M1_TO_20240625_020554.mff
Participant number: 1525-1024
Participant relation: M1
family code out: 1024
Participant ID input: 1525-1024_M1
Participant ID output: 1024M1
1024M1
1024M1 has already been processed
Current data file: /project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1093_P/Q1K_HSJ_1525-1093_P_TO_20241213_094147.mff
Participant number: 1525-1093
Participant relation: P
family code out: 1093
Participant ID input: 1525-1093_P
Participant ID output: 1093P
1093P
1093P has already been processed
Current data file: /project/def-emayada/q1k/experimental/sourcedata/HSJ/eeg/Q1K_HSJ_1525-1106_F1/Q1K_HSJ_1525-1106_F1_TO_20241122_115349.mff
Participant number: 1525-1106
Participant relation: F1
family code out: 1106
Participant ID input: 1525-1106_F1
Participant ID output: 1106F1
1106F1
1106F1 has already been processed


### Visualize all reports at once

In [ ]:
#for file in glob.glob("./../../sourcefiles/EEG/Q1K*/Q1K" + "*AEP" + '*.mff'):
#    # Select anything after the Q1K and before the AEP
#    subject_id = file.split('\\')[1]
#    # Skip sessions that have already been processed
#    print(subject_id)
#    if subject_id in processed_sessions:
#        print (subject_id + ' has already been processed')
#        continue    

#    # Handle participants with error 
#    try:
#        %run -i ./q1k_generate_individual_reports.ipynb --subject_id={subject_id}
#    except Exception as e:
#    # Handle the error (e.g., log it, skip participant, etc.)
#        print(f"Error while reading raw data for {subject_id}: {e}")
